# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

In [52]:
####################### Imports ##############################

##### Infrastructure #######
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectKBest

####### Models ########
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression


In [18]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [19]:
# Parse the cleaned novels. This can take a bit.
start_time = time.time()
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

print("-- Execution time: %s seconds ---" % (time.time() - start_time))

-- Execution time: 17.780986785888672 seconds ---


In [20]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [21]:
# Bag of Words
# Utility function to create a list of the 1000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(1000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    start_time = time.time()
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 100 == 0:
            print("Processing row {} - {} seconds".format(i, (time.time() - start_time)))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [22]:
# Create our data frame with features. This can take a while to run.
start_time = time.time()
word_counts = bow_features(sentences, common_words)
print("-- Execution time: %s seconds ---" % (time.time() - start_time))
word_counts.head()

Processing row 0 - 4.768096923828125 seconds
Processing row 100 - 183.96259808540344 seconds
Processing row 200 - 334.0793879032135 seconds
Processing row 300 - 484.36054706573486 seconds
Processing row 400 - 632.034982919693 seconds
Processing row 500 - 770.7359728813171 seconds
Processing row 600 - 911.4912052154541 seconds
Processing row 700 - 1068.222242116928 seconds
Processing row 800 - 1216.0588638782501 seconds
Processing row 900 - 1339.7893288135529 seconds
Processing row 1000 - 1484.6506931781769 seconds
Processing row 1100 - 1646.1431584358215 seconds
Processing row 1200 - 1795.431675195694 seconds
Processing row 1300 - 1906.8292698860168 seconds
Processing row 1400 - 2034.8620300292969 seconds
Processing row 1500 - 2170.50661945343 seconds
Processing row 1600 - 2306.1058468818665 seconds
Processing row 1700 - 2506.838632106781 seconds
Processing row 1800 - 2708.853232383728 seconds
Processing row 1900 - 2915.0913486480713 seconds
Processing row 2000 - 3148.1815054416656 sec

,happiness,seriously,manner,oop,claws,passion,oh,charles,civility,wrong,...,dine,buildings,bite,afford,finger,butter,witness,recommendation,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [27]:
len(sentences)

5318

In [43]:
# The word counts dataframe will have source, sentence and all the words. Need to add other features to this:
# Number of words in sentence
# Number of unique words in sentence
def number_of_words(sentence):
    example_words = [token for token in sentence if not token.is_punct]
    unique_words = set([token.text for token in example_words])
    return len(unique_words)
    

In [44]:
# Add Unique words
new_df = word_counts
new_df['Unique_Words'] = new_df['text_sentence'].apply(lambda x: number_of_words(x))

In [54]:
# Split the data
Y = new_df['text_source']
X = new_df.drop(['text_sentence','text_source'], 1)

In [55]:
# Select The Best Features
start_time = time.time()
selector = SelectKBest(f_classif, k=50)
selector.fit(X,Y)

idxs_selected = selector.get_support(indices=True)
best_features = X[X.columns[idxs_selected]]
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.5059971809387207 seconds ---


In [56]:
# Do the Test Train Split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.3,
                                                    random_state=0)

In [48]:
# Check the balance
Y.value_counts()

Austen     3649
Carroll    1669
Name: text_source, dtype: int64

In [57]:
################ Logistic Regression ######################
start_time = time.time()
parameters = {
                'penalty':['l1','l2'],
                'C':[0.001,0.01,0.1,1,10,100,10000,100000,100000000]
               
              }

lr = LogisticRegression(class_weight='balanced')

grid = GridSearchCV(lr, parameters, scoring='accuracy', cv=5, verbose=0)
#Fit the Data
grid.fit(X_train, y_train)
print(grid.score(X_test, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))


0.906641604010025
-- Execution time: 72.38328790664673 seconds ---


In [ ]:
################ Gradient Boost #############################
start_time = time.time()
parameters = {'subsample':[0.95],
              'max_depth':[2,4],
              'loss':['exponential'],
             'n_estimators':[500]}

# Initialize the model.
clf = ensemble.GradientBoostingClassifier()

#Create grid and perform 8 cross validation
gradient_grid = GridSearchCV(clf, parameters, cv=5, verbose=0)

#Fit the Data
gradient_grid.fit(X_train, y_train)
print(gradient_grid.score(X_test, y_test))
print("--- %s seconds ---" % (time.time() - start_time))

In [60]:
print(gradient_grid.score(X_test, y_test))

0.8859649122807017
